In [9]:
from music21 import *

if False: # if using colab
    from google.colab import drive
    !add-apt-repository -y ppa:mscore-ubuntu/mscore3-stable
    !apt-get update
    !apt-get install musescore3
if False: # windows vs code
    !pip install -U music21
    !pip install ms3

In [10]:
import collections
import datetime
import glob
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Optional

# Data

In [11]:

# PATH FOR DATA: '/content/drive/MyDrive/ME539 - Music Generation/data' maestro-v3.0.0-midi/maestro-v3.0.0/2018/MIDI-Unprocessed_Schubert10-12_MID--AUDIO_20_R2_2018_wav

data_dir = pathlib.Path('data/maestro-v2.0.0')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'maestro-v2.0.0-midi.zip',
      origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )
filenames = glob.glob(str(data_dir/'**/*.mid*'))
print('Number of files:', len(filenames))

sample_file = filenames[1]
print(sample_file)

def midi_to_list(file):
  score = converter.parse(file, quantize = False)
  # score.plot()
  # Access the stream containing notes and rests
  notes_and_rests_stream = score.flat.notesAndRests


  # Take all elements in note stream and convert to list containing pitch, note length, and position.
  for element in notes_and_rests_stream:
      if element.isNote:
          all_notes_list.append([element.pitch.midi,element.duration.quarterLength,element.offset])  # This will add note value, note length, and note position to the list.
all_notes_list = []

#TODO: Implement random selection of files to be loaded
for f in filenames[1:20]: # load x amount of MIDI files into the list of notes
  midi_to_list(f)



59243107/59243107 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Number of files: 1282
data\maestro-v2.0.0\2004\MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_06_Track06_wav.midi


c:\Users\ruiza\AppData\Local\Programs\Python\Python310\lib\site-packages\music21\stream\base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


In [14]:
# Convert output back into MIDI

TEMPO = 120

output_stream = stream.Stream() # create enmpty music21 data stream
tempo_mark = tempo.MetronomeMark(number=TEMPO)
output_stream.append(tempo_mark) # set stream tempo


for notes in all_notes_list:
	new_note = note.Note(notes[0]) # creates new note object based on pitch from notes_list
	new_note.offset = notes[2] # assigns note offset or position
	new_note.duration.quarterLength = notes[1] # assigns note duration
	output_stream.insert(new_note) # inserts note into output_stream

output_stream.plot()

# LSTM Model

In [21]:
# Parameters:
# Vocab size 128
# Start with sequence size of 50 notes
# Batch size of 50 examples

sequence_length = 50
vocab_size = 128

input_shape = (sequence_length, 3)
learning_rate = 0.005

model_input = tf.keras.Input(input_shape)
